In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D, Lambda, MaxPooling2D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate, Reshape, GRU
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Conv2D, SpatialDropout1D
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.utils import to_categorical
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [2]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
X_train = X_tr[0].astype(str).tolist()
X_test = X_t[0].astype(str).tolist()


Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [3]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [17]:
embedding_dim=300
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    inp=Input(shape=(max_len,))

    embed=Embedding(input_dim=max_word,output_dim=embedding_dim,input_length=max_len)(inp)
    spadrop=SpatialDropout1D(0.3)(embed)
    gru=GRU(300)(spadrop)
    dense1=Dense(128,activation='relu')(gru)
    drop=Dropout(0.2)(dense1)
    outp=Dense(3,activation='softmax')(drop)
    model=Model(inputs=inp,outputs=outp)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision,Recall])
    return model

Chạy mô hình với data train

In [18]:
model=model1()
model.summary()
history=model.fit(tr_x, tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_6 (Embedding)         │ (None, 50, 300)        │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_6             │ (None, 50, 300)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 300)            │       541,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,580,715 (25.10 MB)

 Trainable params: 6,580,715 (25.10 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 35s 96ms/step - accuracy: 0.5285 - loss: 0.5551 - precision_1: 0.6113 - recall_1: 0.3356 - val_accuracy: 0.7244 - val_loss: 0.4072 - val_precision_1: 0.7367 - val_recall_1: 0.7117
Epoch 2/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 33s 97ms/step - accuracy: 0.7833 - loss: 0.3363 - precision_1: 0.7962 - recall_1: 0.7691 - val_accuracy: 0.7259 - val_loss: 0.4066 - val_precision_1: 0.7338 - val_recall_1: 0.7146
Epoch 3/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 33s 97ms/step - accuracy: 0.8608 - loss: 0.2319 - precision_1: 0.8688 - recall_1: 0.8542 - val_accuracy: 0.7044 - val_loss: 0.4598 - val_precision_1: 0.7138 - val_recall_1: 0.6946
Epoch 4/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 37s 107ms/step - accuracy: 0.9083 - loss: 0.1604 - precision_1: 0.9129 - recall_1: 0.9055 - val_accuracy: 0.7000 - val_loss: 0.5357 - val_precision_1: 0.7067 - val_recall_1: 0.6944
Epoch 5/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 35s 102ms/step - accuracy: 0.9327 - loss: 0.1200 - precision_1: 0.9361 - recall

Thử dữ liệu với data test

In [19]:
pred_label=model.predict(t_x)
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))


111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
              precision    recall  f1-score   support

           0       0.66      0.63      0.64      1001
           1       0.62      0.65      0.64      1430
           2       0.72      0.70      0.71      1103

    accuracy                           0.66      3534
   macro avg       0.67      0.66      0.66      3534
weighted avg       0.66      0.66      0.66      3534

